In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
%load_ext blackcellmagic

import pyphi
from visualize_pyphi import utils
from visualize_pyphi import network_generator
from visualize_pyphi import fill_slots
import string
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import itertools
import toolz
from pyphi.models.subsystem import FlatCauseEffectStructure as sep

'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'IIT_4.0_SMALL_PHI'
IIT_VERSION = 'maximal-state-first'
PARTITION_TYPE = 'TRI'
'''
pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


('IIT_4.0_SMALL_PHI', 'maximal-state-first', 'TRI')

# Activation and weight function parameters

In [31]:
# Sigmoid formula is y = l / (l + np.e ** (-k * (x - x0)))
# Sigmoid values
l = 1
x0 = .5
k = 3.8

In [32]:
# Weight decay
decay_vals = np.linspace(1, 0, 200)

weight_vals = []
for d in decay_vals:
    weights = [d ** n for n in range(0, 4)]
    weight_vals.append(weights)

In [33]:
decay_val = 198
d = weight_vals[decay_val]

### 3 nodes

In [23]:
weights3node = [
    weight_vals[decay_val][0],
    weight_vals[decay_val][1],
    weight_vals[decay_val][1],
]
weights3node = weights3node / sum(weights3node)
print(weights3node,'\nsum =',sum(weights3node))

[0.99004975 0.00497512 0.00497512] 
sum = 1.0000000000000002


In [24]:
# s = strong, m = medium, w = weak
s = weights3node[0]
m = weights3node[1]
s,m,w

(0.9900497512437813, 0.004975124378109475, 2.4999375015624835e-05)

In [25]:
net_name3 = "bigphi3"

# 4 nodes toroidal grid, each node has 1 strong (self-loop), 2 medium (nearest neighbors) and one weak (far neighbor)
weights = np.array(
    [
        [s, m, m],  # A
        [m, s, m],  # B
        [m, s, m],  # C
        #A, B, C
    ]
)

node_labels = [string.ascii_uppercase[n] for n in range(len(weights))]
mech_func = ['l' for n in range(len(weights))] # "l" stands for LogFunc

network = network_generator.get_net(
    mech_func,
    weights,
    l=l,
    k=k,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name4,
    pickle_network=False,
)

state = (0, 0, 0)
subsystem3 = pyphi.Subsystem(network, state)

  0%|          | 0/8 [00:00<?, ?it/s]

In [26]:
ces3 = fill_slots.fill_slots_reflexive(subsystem3)
# utils.ces2df(ces4,subsystem4)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [27]:
# To compute relations from scratch
relations3 = pyphi.relations.relations(subsystem3,sep(ces3))

  0%|          | 0/339 [00:00<?, ?it/s]

In [28]:
# Compute big phi
sia3 = pyphi.big_phi.sia(subsystem3,ces3,relations3)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [29]:
print(f'{net_name3}\nkrels=all\n#rels={len(sia3.phi_structure.relations)}\nbigphi={sia3.phi}')
utils.ces2df(sia3.phi_structure.distinctions,subsystem3)

bigphi3
krels=all
#rels=339
bigphi=3.251937332271986


,mechanism,state,cause,state,phi,effect,state,phi
0,A,0,A,0,0.690120,A,0,0.690120
1,B,0,B,0,0.717256,B,0,0.408509
2,C,0,C,0,0.000910,C,0,0.001577
3,AB,00,BC,00,0.001145,BC,00,0.002148
4,AC,00,AC,00,0.000985,AC,00,0.002721
5,ABC,000,ABC,000,0.000884,ABC,000,0.004717


### 4 nodes

In [6]:
weights4node = [
    weight_vals[decay_val][0],
    weight_vals[decay_val][1],
    weight_vals[decay_val][1],
    weight_vals[decay_val][2],
]
weights4node = weights4node / sum(weights4node)
print(weights4node,'\nsum =',sum(weights4node))

[9.90025e-01 4.97500e-03 4.97500e-03 2.50000e-05] 
sum = 1.0000000000000004


In [7]:
# s = strong, m = medium, w = weak
s = weights4node[0]
m = weights4node[1]
w = weights4node[3]
s,m,w

(0.9900250000000003, 0.004975000000000023, 2.5000000000000225e-05)

In [8]:
net_name4 = "bigphi4"

# 4 nodes toroidal grid, each node has 1 strong (self-loop), 2 medium (nearest neighbors) and one weak (far neighbor)
weights = np.array(
    [
        [s, m, w, m],  # A
        [m, s, m, w],  # B
        [w, m, s, m],  # C
        [m, w, m, s],  # D
        #A, B, C, D
    ]
)

node_labels = [string.ascii_uppercase[n] for n in range(len(weights))]
mech_func = ['l' for n in range(len(weights))] # "l" stands for LogFunc

network = network_generator.get_net(
    mech_func,
    weights,
    l=l,
    k=k,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name4,
    pickle_network=False,
)

state = (0, 0, 0, 0)
subsystem4 = pyphi.Subsystem(network, state)

  0%|          | 0/16 [00:00<?, ?it/s]

In [9]:
ces4 = fill_slots.fill_slots_reflexive(subsystem4)
# utils.ces2df(ces4,subsystem4)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
# To compute relations from scratch
relations4 = pyphi.relations.relations(subsystem,sep(ces),max_degree=2)

In [11]:
# #To load precomputed relations
# relations4 = utils.decompress_relations(subsystem4, ces4, f"relations/{net_name4}_*")

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/65138 [00:00<?, ?it/s]

In [12]:
# Compute big phi
sia4 = pyphi.big_phi.sia(subsystem4,ces4,relations4)

2022-02-14 10:35:58,105	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8269


Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [13]:
print(f'{net_name4}\nkrels=all\n#rels={len(sia4.phi_structure.relations)}\nbigphi={sia4.phi}')
utils.ces2df(sia4.phi_structure.distinctions,subsystem4)

bigphi4
krels=all
#rels=65138
bigphi=522.5734344627173


,mechanism,state,cause,state,phi,effect,state,phi
0,A,0,A,0,0.690108,A,0,0.690108
1,B,0,B,0,0.690108,B,0,0.690108
2,C,0,C,0,0.690108,C,0,0.690108
3,D,0,D,0,0.690108,D,0,0.690108
4,AB,00,AB,00,0.000746,AB,00,0.002721
5,AC,00,AC,00,0.000004,AC,00,0.000014
6,AD,00,AD,00,0.000746,AD,00,0.002721
7,BC,00,BC,00,0.000746,BC,00,0.002721
8,BD,00,BD,00,0.000004,BD,00,0.000014
9,CD,00,CD,00,0.000746,CD,00,0.002721


### 5 nodes

In [14]:
weights5node = [
    weight_vals[decay_val][0],
    weight_vals[decay_val][1],
    weight_vals[decay_val][1],
    weight_vals[decay_val][2],
    weight_vals[decay_val][2]    
]
weights5node = weights5node / sum(weights5node)
print(weights5node,'\nsum =',sum(weights5node))

[9.90000250e-01 4.97487563e-03 4.97487563e-03 2.49993750e-05
 2.49993750e-05] 
sum = 1.0000000000000002


In [15]:
# s = strong, m = medium, w = weak
s = weights5node[0]
m = weights5node[1]
w = weights5node[3]
s,m,w

(0.9900002499937505, 0.004974875628109321, 2.4999375015624835e-05)

In [16]:
net_name5 = "bigphi5"

# 5 nodes toroidal grid, each node has 1 strong (self-loop), 2 medium (nearest neighbors) and two weak (far neighbors)
weights = np.array(
    [
        [s, m, w, w, m],  # A
        [m, s, m, w, w],  # B
        [w, m, s, m, w],  # C
        [w, w, m, s, m],  # D
        [m, w, w, m, s],  # E
        #A, B, C, D, E
    ]
)

node_labels = [string.ascii_uppercase[n] for n in range(len(weights))]
mech_func = ['l' for n in range(len(weights))] # "l" stands for LogFunc

network = network_generator.get_net(
    mech_func,
    weights,
    l=l,
    k=k,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name5,
    pickle_network=False,
)

state = (0, 0, 0, 0, 0)
subsystem5 = pyphi.Subsystem(network, state)

  0%|          | 0/32 [00:00<?, ?it/s]

In [18]:
ces5 = fill_slots.fill_slots_reflexive(subsystem5)
# utils.ces2df(ces5,subsystem5)

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

In [19]:
# To compute relations from scratch
relations5 = pyphi.relations.relations(subsystem,sep(ces),max_degree=2)

In [20]:
# #To load precomputed relations
# r_degree = 4
# relations5 = utils.decompress_relations(subsystem5, ces5, f"relations/{net_name5}_krels_{r_degree}*")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/59225 [00:00<?, ?it/s]

In [21]:
# Compute big phi
sia5 = pyphi.big_phi.sia(subsystem5,ces5,relations5)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state


In [22]:
print(f'{net_name5}\nkrels={r_degree}\n#rels={len(sia5.phi_structure.relations)}\nbigphi={sia5.phi}')
utils.ces2df(sia5.phi_structure.distinctions,subsystem5)

bigphi5
krels=4
#rels=59225
bigphi=0.02005838021693895


,mechanism,state,cause,state,phi,effect,state,phi
0,A,0,A,0,0.690096,A,0,0.690096
1,B,0,B,0,0.690096,B,0,0.690096
2,C,0,C,0,0.690096,C,0,0.690096
3,D,0,D,0,0.690096,D,0,0.690096
4,E,0,E,0,0.690096,E,0,0.690096
5,AB,00,AB,00,0.000746,AB,00,0.002721
6,AC,00,AC,00,0.000004,AC,00,0.000014
7,AD,00,AD,00,0.000004,AD,00,0.000014
8,AE,00,AE,00,0.000746,AE,00,0.002721
9,BC,00,BC,00,0.000746,BC,00,0.002721
